In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from wordcloud import WordCloud
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import os 

k_colors = ['rgb(0,127,206)','rgb(62,196,4)','rgb(255,125,16)','rgb(253,99,90)']#blue,gree,orange,red
plotly_themes = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]#plotly_white plotly_dark
theme = plotly_themes[1]


## Reading the data

In [3]:
#path to data files
notebook_path = os.getcwd()
project_path = notebook_path[0:len(notebook_path)-25]+"data/"
project_code_path = notebook_path[0:len(notebook_path)-25]+"code/"

In [4]:
movies = pd.read_csv(project_path+'movie.csv')
movies.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [5]:
tag = pd.read_csv(project_path+'tag.csv')
tag.head(1)

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40


In [6]:
ratings = pd.read_csv(project_path+'rating.csv')
ratings["year_month"]=pd.DatetimeIndex(ratings['timestamp']).year*100+\
pd.DatetimeIndex(ratings['timestamp']).month
ratings.head(1)

,userId,movieId,rating,timestamp,year_month
0,1,2,3.5,2005-04-02 23:53:47,200504


For time porpuse, we will have the information of the last 2 years

In [7]:
ratings = ratings.loc[ratings.timestamp>'2013-01-01']
ratings.to_csv("../../data/ratings_filtered.csv",index = False)

## Cleaning the year from the tittle movies

In [8]:
movies["year"] = movies["title"].str.extract('(\(\d\d\d\d\))', expand=False)
movies['year'] = movies["year"].str.extract('(\d\d\d\d)', expand=False)  # only numbers
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')  # replace numbers and parenthesis
movies['title'] = movies['title'].apply(lambda x: x.strip())

movies['genre_list'] = movies['genres'].str.split('|').tolist()
flat_genre = [item for sublist in movies['genre_list']for item in sublist]  # flatten the list
set_genre = set(flat_genre)  # convert to a set to make unique
unique_genre = list(set_genre)  # back to list
# remove NA
unique_genre.remove("(no genres listed)")

# create columns by each unique genre
movies = movies.reindex(
movies.columns.tolist() + unique_genre, axis=1, fill_value=0)

# for each value inside column, update the dummy
for index, row in movies.iterrows():
    for val in row["genres"].split('|'):
        if val != "(no genres listed)":
            movies.loc[index, val] = 1

movies.head(3)

,movieId,title,genres,year,genre_list,Animation,Documentary,Adventure,Romance,Comedy,...,Children,Action,Western,Fantasy,Musical,Horror,Mystery,War,Thriller,Film-Noir
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,"[Adventure, Animation, Children, Comedy, Fantasy]",1,0,1,0,1,...,1,0,0,1,0,0,0,0,0,0
1,2,Jumanji,Adventure|Children|Fantasy,1995,"[Adventure, Children, Fantasy]",0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
2,3,Grumpier Old Men,Comedy|Romance,1995,"[Comedy, Romance]",0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
movies.dtypes

movieId         int64
title          object
genres         object
year           object
genre_list     object
Animation       int64
Documentary     int64
Adventure       int64
Romance         int64
Comedy          int64
Crime           int64
Drama           int64
IMAX            int64
Sci-Fi          int64
Children        int64
Action          int64
Western         int64
Fantasy         int64
Musical         int64
Horror          int64
Mystery         int64
War             int64
Thriller        int64
Film-Noir       int64
dtype: object

## History variables from the rates of the movies

Lets create the mean rating and count from the movies in the last 2 years

In [10]:
movies_ratings=ratings.groupby(["movieId","year_month"]).agg({"userId":"count",
                                "rating": "mean"})\
.reset_index()
movies_ratings.columns = ["movieId","year_month","count","mean_rating"]
movies_ratings.head(1)

,movieId,year_month,count,mean_rating
0,1,201301,124,3.971774


With the mean rating lets create the history of movie ratings: 

* Last month mean movie rating
* Last month count movie rating
* Mean of the last 3 months of the rate of the movies
* Mean of the last 6 months of the rate of the movies
* The ratio of the last month mean movie rating and the mean of the last 3 months of the rate of the movies
* The ratio of the last month mean movie rating and the mean of the last 6 months of the rate of the movies

In [76]:
movies_ratings["last_1_months_movie_rating"] = movies_ratings.groupby("movieId")['mean_rating'].shift(1)\
.reset_index()["mean_rating"]
movies_ratings["last_1_months_movie_rating_counts"] = movies_ratings.groupby("movieId")['count'].shift(1)\
.reset_index()["count"]
movies_ratings["mean_3_months_movie_rating"] = movies_ratings.groupby("movieId")['last_1_months_movie_rating'].rolling(3).mean()\
.reset_index()["last_1_months_movie_rating"]
movies_ratings["mean_6_months_movie_rating"] = movies_ratings.groupby("movieId")['last_1_months_movie_rating'].rolling(6).mean()\
.reset_index()["last_1_months_movie_rating"]
movies_ratings["last_1_month_per_3_months_movie_rating"] = movies_ratings.last_1_months_movie_rating/movies_ratings.mean_3_months_movie_rating
movies_ratings["last_1_month_per_6_months_movie_rating"] = movies_ratings.last_1_months_movie_rating/movies_ratings.mean_6_months_movie_rating
movies_ratings = movies_ratings.loc[:,~movies_ratings.columns.isin(["count","mean_rating"])]
movies_ratings.head(10)

,movieId,year_month,last_1_months_movie_rating,last_1_months_movie_rating_counts,mean_3_months_movie_rating,mean_6_months_movie_rating,last_1_month_per_3_months_movie_rating,last_1_month_per_6_months_movie_rating
0,1,201301,NaN,NaN,NaN,NaN,NaN,NaN
1,1,201302,3.971774,124.0,NaN,NaN,NaN,NaN
2,1,201303,4.033654,104.0,NaN,NaN,NaN,NaN
3,1,201304,3.974227,97.0,3.993218,NaN,0.995244,NaN
4,1,201305,3.954082,98.0,3.987321,NaN,0.991664,NaN
5,1,201306,3.825243,103.0,3.917850,NaN,0.976363,NaN
6,1,201307,3.971591,88.0,3.916972,3.955095,1.013944,1.004171
7,1,201308,4.100000,105.0,3.965611,3.976466,1.033889,1.031066
8,1,201309,3.771429,70.0,3.947673,3.932762,0.955355,0.958977
9,1,201310,3.818841,138.0,3.896756,3.906864,0.980005,0.977470


## History variables from the ratings of the genres of movies

In [12]:
movies_genre_gather = movies[["movieId","genre_list"]].explode("genre_list")

movies_ratings_genre=movies_ratings.merge(movies_genre_gather,how = "left")

genre_ratings=movies_ratings_genre.groupby(["genre_list","year_month"]).agg({"movieId":"count",
                                "mean_rating": "mean"})\
.reset_index()

genre_ratings.columns = ["genre_list","year_month","count","mean_rating"]

genre_ratings.loc[genre_ratings.genre_list=="Drama"].head(5)

,genre_list,year_month,count,mean_rating
204,Drama,201301,3638,3.520814
205,Drama,201302,3256,3.547158
206,Drama,201303,3158,3.628895
207,Drama,201304,3531,3.351295
208,Drama,201305,3543,3.448825


With the mean rating lets create the history of genre ratings: 

* Last month mean genre rating
* Last month count genre rating
* Mean of the last 3 months of the rate of the genre
* Mean of the last 6 months of the rate of the genre
* The ratio of the last month mean genre rating and the mean of the last 3 months of the rate of the genre
* The ratio of the last month mean genre rating and the mean of the last 6 months of the rate of the genre

In [13]:
genre_ratings["last_1_months_genre_rating"] = genre_ratings.groupby("genre_list")['mean_rating'].shift(1)\
.reset_index()["mean_rating"]
genre_ratings["last_1_months_genre_rating_counts"] = genre_ratings.groupby("genre_list")['count'].shift(1)\
.reset_index()["count"]
genre_ratings["mean_3_months_genre_rating"] = genre_ratings.groupby("genre_list")['last_1_months_genre_rating'].rolling(3).mean()\
.reset_index()["last_1_months_genre_rating"]
genre_ratings["mean_6_months_genre_rating"] = genre_ratings.groupby("genre_list")['last_1_months_genre_rating'].rolling(6).mean()\
.reset_index()["last_1_months_genre_rating"]
genre_ratings["last_1_month_per_3_months_genre_rating"] = genre_ratings.last_1_months_genre_rating/genre_ratings.mean_3_months_genre_rating
genre_ratings["last_1_month_per_6_months_genre_rating"] = genre_ratings.last_1_months_genre_rating/genre_ratings.mean_6_months_genre_rating

genre_ratings.loc[genre_ratings.genre_list=="Drama"].head(7)

,genre_list,year_month,count,mean_rating,last_1_months_genre_rating,last_1_months_genre_rating_counts,mean_3_months_genre_rating,mean_6_months_genre_rating,last_1_month_per_3_months_genre_rating,last_1_month_per_6_months_genre_rating
204,Drama,201301,3638,3.520814,NaN,NaN,NaN,NaN,NaN,NaN
205,Drama,201302,3256,3.547158,3.520814,3638.0,NaN,NaN,NaN,NaN
206,Drama,201303,3158,3.628895,3.547158,3256.0,NaN,NaN,NaN,NaN
207,Drama,201304,3531,3.351295,3.628895,3158.0,3.565622,NaN,1.017745,NaN
208,Drama,201305,3543,3.448825,3.351295,3531.0,3.509116,NaN,0.955025,NaN
209,Drama,201306,3064,3.367797,3.448825,3543.0,3.476338,NaN,0.992086,NaN
210,Drama,201307,3054,3.632905,3.367797,3064.0,3.389305,3.477464,0.993654,0.968463


In [14]:
#now we will paste the genre variables
genre_variables = ["genre_list","year_month","last_1_months_genre_rating","mean_3_months_genre_rating",
                  "mean_6_months_genre_rating","last_1_month_per_3_months_genre_rating",
                  "last_1_month_per_6_months_genre_rating"]

genre_ratings_to_join=genre_ratings[genre_ratings.year_month >= 201403][genre_variables]

In [15]:
genre_ratings_to_join.loc[genre_ratings_to_join.genre_list=="Drama"]

,genre_list,year_month,last_1_months_genre_rating,mean_3_months_genre_rating,mean_6_months_genre_rating,last_1_month_per_3_months_genre_rating,last_1_month_per_6_months_genre_rating
218,Drama,201403,3.567098,3.538904,3.542391,1.007967,1.006975
219,Drama,201404,3.497814,3.541073,3.538563,0.987784,0.988484
220,Drama,201405,3.304345,3.456419,3.494073,0.956003,0.945700
221,Drama,201406,3.538060,3.446740,3.492822,1.026495,1.012952
222,Drama,201407,3.374054,3.405486,3.473280,0.990770,0.971432
223,Drama,201408,3.595230,3.502448,3.479434,1.026491,1.033280
224,Drama,201409,3.516486,3.495257,3.470998,1.006074,1.013105
225,Drama,201410,3.510313,3.540676,3.473081,0.991424,1.010720
226,Drama,201411,3.528179,3.518326,3.510387,1.002800,1.005068
227,Drama,201412,3.530950,3.523147,3.509202,1.002215,1.006197


Lets paste the genre variables to the movies and get the mean of the history variables for each of the genres that are asociate to the movies

In [16]:
movies_genre_ratings=movies_genre_gather.merge(genre_ratings_to_join,how = "left")
movies_genre_ratings_agg =movies_genre_ratings.groupby(["movieId","year_month"]).mean()\
.reset_index()

In [17]:
movies_genre_ratings_agg.loc[movies_genre_ratings_agg["movieId"]==1]

,movieId,year_month,last_1_months_genre_rating,mean_3_months_genre_rating,mean_6_months_genre_rating,last_1_month_per_3_months_genre_rating,last_1_month_per_6_months_genre_rating
0,1,201403,3.312358,3.291977,3.328847,1.006089,0.994983
1,1,201404,3.205175,3.252705,3.304522,0.985209,0.969793
2,1,201405,3.014464,3.177332,3.247004,0.948812,0.928399
3,1,201406,3.301486,3.173708,3.232843,1.040576,1.021599
4,1,201407,3.327976,3.214642,3.233674,1.035065,1.029122
5,1,201408,3.348829,3.326097,3.251715,1.006784,1.029834
6,1,201409,3.318001,3.331602,3.252655,0.995730,1.019810
7,1,201410,3.311240,3.326023,3.270333,0.995735,1.012570
8,1,201411,3.385594,3.338278,3.332188,1.014164,1.015916
9,1,201412,3.176153,3.290996,3.311299,0.964888,0.958950


## History variables from the ratings of the tags of movies

This is the variable that I had to think the most, because each tag that a user gives to a movie can be applied only in one month and for the next periods of time it could complete disappear. To deal with this, I decided to calculate the  mean of the rating given to each tag  and then calculate the weighted mean of the tags to the movies on the last 6 months. This approach gives more weight to the tags that were used by more users than the ones that were sporadically used.

For time porpuse, we will have the information of the last 2 years

In [18]:
tag["year_month"] = pd.DatetimeIndex(tag['timestamp']).year*100+\
pd.DatetimeIndex(tag['timestamp']).month
tag = tag.loc[tag.timestamp>'2013-01-01']

### Cleaning the tags

In [19]:
#if not installed, nltk.download(). look for corpora>stopwords, packages>wordnet
try:
    stop_words = set(stopwords.words("english")) 
except LookupError:
    import nltk
    nltk.download()

In [20]:
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)#only alfanumeric and spaces
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text] # meeting > meet, was > be
    text = [word for word in text if not word in stop_words] #exclude stop words
    text = " ".join(text)
    return text

tag['processed_tag'] = tag.tag.apply(lambda x: clean_text(x))

In [21]:
tag

,userId,movieId,tag,timestamp,year_month,processed_tag
1,65,208,dark hero,2013-05-10 01:41:18,201305,dark hero
2,65,353,dark hero,2013-05-10 01:41:19,201305,dark hero
3,65,521,noir thriller,2013-05-10 01:39:43,201305,noir thriller
4,65,592,dark hero,2013-05-10 01:41:18,201305,dark hero
5,65,668,bollywood,2013-05-10 01:37:56,201305,bollywood
...,...,...,...,...,...,...
465558,138446,7164,visually appealing,2013-01-23 23:30:55,201301,visually appeal
465559,138446,55999,dragged,2013-01-23 23:29:32,201301,drag
465560,138446,55999,Jason Bateman,2013-01-23 23:29:38,201301,jason bateman
465561,138446,55999,quirky,2013-01-23 23:29:38,201301,quirky


In [22]:
tag.to_csv("../../data/tag_filtered.csv",index = False)

Lets merge the tags and the ratings on the movies

In [23]:
tags_ratings=tag.loc[:,~tag.columns.isin(["userId","timestamp","tag"])]\
.drop_duplicates()\
.merge(ratings,"left")

In [24]:
tags_ratings

,movieId,year_month,processed_tag,userId,rating,timestamp
0,208,201305,dark hero,1141.0,3.5,2013-05-08 12:12:27
1,208,201305,dark hero,4912.0,3.0,2013-05-03 13:48:33
2,208,201305,dark hero,13920.0,2.5,2013-05-01 19:07:45
3,208,201305,dark hero,22377.0,3.5,2013-05-16 20:43:17
4,208,201305,dark hero,25591.0,1.5,2013-05-29 20:04:13
...,...,...,...,...,...,...
3745555,55999,201301,sad,25237.0,3.5,2013-01-07 01:34:28
3745556,55999,201301,sad,30376.0,2.5,2013-01-03 23:20:00
3745557,55999,201301,sad,32514.0,3.0,2013-01-13 23:23:46
3745558,55999,201301,sad,114651.0,4.0,2013-01-18 18:09:21


Lets agregate the mean rating for all the processed tags

In [25]:
tags_ratings_agg = tags_ratings.groupby(["processed_tag","year_month"]).agg({"userId":"count","rating":"mean"})\
.reset_index()
tags_ratings_agg.columns = ["processed_tag","year_month","count","rating"]

In [26]:
tags_ratings_agg.loc[tags_ratings_agg["processed_tag"]=="toy"]

,processed_tag,year_month,count,rating
44485,toy,201301,69,3.862319
44486,toy,201302,67,3.880597
44487,toy,201306,1,3.000000
44488,toy,201310,32,3.156250
44489,toy,201312,2,1.750000
44490,toy,201403,29,3.431034
44491,toy,201407,46,3.847826
44492,toy,201501,160,3.815625
44493,toy,201502,157,3.815287


lets get reed of the nulls ratings when we made the left join of the tags

In [27]:
tags_ratings_agg_notNull =  tags_ratings_agg.loc[tags_ratings_agg.rating.notnull()]

Because of a tag can be use in one month and not in other, lets create a cross join of every posible tag and months.

In [28]:
unique_cleaned_tags =tags_ratings_agg_notNull.processed_tag.drop_duplicates().reset_index(drop = True)
unique_year_months = tags_ratings_agg_notNull.year_month.drop_duplicates().reset_index(drop = True)

df1 = pd.DataFrame({'processed_tag':unique_cleaned_tags})
df2 = pd.DataFrame({'year_month':unique_year_months})
df1['tmp'] = 1
df2['tmp'] = 1

unique_months_years_tags = df1.merge(df2)
unique_months_years_tags

,processed_tag,tmp,year_month
0,,1,201301
1,,1,201302
2,,1,201303
3,,1,201306
4,,1,201309
...,...,...,...
380938,özgür yildirim,1,201408
380939,özgür yildirim,1,201307
380940,özgür yildirim,1,201304
380941,özgür yildirim,1,201308


If we are going to take the weighted mean of the rating, we will substitue 0 with a small number 0.001 when joining how many users used the tag

In [29]:
tags_ratings_agg_notNull_full=unique_months_years_tags.merge(tags_ratings_agg_notNull,"left").fillna(0.0001).\
sort_values(by=['processed_tag', 'year_month']).\
reset_index(drop = True)

In [30]:
tags_ratings_agg_notNull_full.loc[tags_ratings_agg_notNull_full['processed_tag']=='toy']

,processed_tag,tmp,year_month,count,rating
351027,toy,1,201301,69.0000,3.862319
351028,toy,1,201302,67.0000,3.880597
351029,toy,1,201303,0.0001,0.000100
351030,toy,1,201304,0.0001,0.000100
351031,toy,1,201305,0.0001,0.000100
351032,toy,1,201306,1.0000,3.000000
351033,toy,1,201307,0.0001,0.000100
351034,toy,1,201308,0.0001,0.000100
351035,toy,1,201309,0.0001,0.000100
351036,toy,1,201310,32.0000,3.156250


Now lets get the lag of the mean rating 

In [31]:
tags_ratings_agg_notNull_full["last_1_months_tag_rating"] = tags_ratings_agg_notNull_full.groupby("processed_tag")["rating"]\
.shift(1)\
.reset_index()["rating"]

tags_ratings_agg_notNull_full["last_1_months_tag_count"] = tags_ratings_agg_notNull_full.groupby("processed_tag")["count"]\
.shift(1)\
.reset_index()["count"]

In [32]:
tags_ratings_agg_notNull_full.loc[tags_ratings_agg_notNull_full["processed_tag"]=='toy']

,processed_tag,tmp,year_month,count,rating,last_1_months_tag_rating,last_1_months_tag_count
351027,toy,1,201301,69.0000,3.862319,NaN,NaN
351028,toy,1,201302,67.0000,3.880597,3.862319,69.0000
351029,toy,1,201303,0.0001,0.000100,3.880597,67.0000
351030,toy,1,201304,0.0001,0.000100,0.000100,0.0001
351031,toy,1,201305,0.0001,0.000100,0.000100,0.0001
351032,toy,1,201306,1.0000,3.000000,0.000100,0.0001
351033,toy,1,201307,0.0001,0.000100,3.000000,1.0000
351034,toy,1,201308,0.0001,0.000100,0.000100,0.0001
351035,toy,1,201309,0.0001,0.000100,0.000100,0.0001
351036,toy,1,201310,32.0000,3.156250,0.000100,0.0001


And calculate the weighted mean in the last 6 months 

In [33]:
tags_ratings_agg_notNull_full["last_6_months_tag_count"] = tags_ratings_agg_notNull_full\
.groupby("processed_tag")["last_1_months_tag_count"].rolling(6).sum()\
.reset_index()["last_1_months_tag_count"]

wm = lambda x: np.average(x, weights=tags_ratings_agg_notNull_full.loc[x.index, "last_1_months_tag_count"])

tags_ratings_agg_notNull_full["last_6_months_weighted_rating"] = tags_ratings_agg_notNull_full\
.groupby("processed_tag")["last_1_months_tag_rating"].rolling(6).apply(wm)\
.reset_index()["last_1_months_tag_rating"]

In [34]:
tags_ratings_agg_notNull_full.loc[tags_ratings_agg_notNull_full["processed_tag"]=='toy']

,processed_tag,tmp,year_month,count,rating,last_1_months_tag_rating,last_1_months_tag_count,last_6_months_tag_count,last_6_months_weighted_rating
351027,toy,1,201301,69.0000,3.862319,NaN,NaN,NaN,NaN
351028,toy,1,201302,67.0000,3.880597,3.862319,69.0000,NaN,NaN
351029,toy,1,201303,0.0001,0.000100,3.880597,67.0000,NaN,NaN
351030,toy,1,201304,0.0001,0.000100,0.000100,0.0001,NaN,NaN
351031,toy,1,201305,0.0001,0.000100,0.000100,0.0001,NaN,NaN
351032,toy,1,201306,1.0000,3.000000,0.000100,0.0001,NaN,NaN
351033,toy,1,201307,0.0001,0.000100,3.000000,1.0000,137.0003,3.864955
351034,toy,1,201308,0.0001,0.000100,0.000100,0.0001,68.0004,3.867624
351035,toy,1,201309,0.0001,0.000100,0.000100,0.0001,1.0005,2.998501
351036,toy,1,201310,32.0000,3.156250,0.000100,0.0001,1.0005,2.998501


In [35]:
columns_tags=["processed_tag","year_month","last_6_months_weighted_rating","last_6_months_tag_count"]
tags_ratings_to_join =  tags_ratings_agg_notNull_full[tags_ratings_agg_notNull_full["year_month"]>=201403][columns_tags]
tags_ratings_to_join.loc[tags_ratings_to_join.processed_tag=="law"]

,processed_tag,year_month,last_6_months_weighted_rating,last_6_months_tag_count
200192,law,201403,0.000100,0.0006
200193,law,201404,0.000100,0.0006
200194,law,201405,0.000100,0.0006
200195,law,201406,3.642597,7.0005
200196,law,201407,3.642597,7.0005
200197,law,201408,4.121187,66.0004
200198,law,201409,4.121187,66.0004
200199,law,201410,4.121187,66.0004
200200,law,201411,4.097205,72.0003
200201,law,201412,4.146128,65.0004


We join to the original tags givien by users

In [36]:
tags_weighted_rating=tag[tag.year_month>=201403].merge(tags_ratings_to_join,"left").\
fillna(0.0001)
tags_weighted_rating

,userId,movieId,tag,timestamp,year_month,processed_tag,last_6_months_weighted_rating,last_6_months_tag_count
0,96,106696,animation,2014-03-29 12:51:30,201403,animation,3.880922,1562.0000
1,96,106696,beautiful,2014-03-29 12:51:37,201403,beautiful,3.915995,869.0000
2,96,106696,characters,2014-03-29 12:51:40,201403,character,3.884374,320.0001
3,96,106696,Disney,2014-03-29 12:51:32,201403,disney,3.825603,539.0000
4,96,106696,feminist,2014-03-29 12:51:42,201403,feminist,3.886350,110.0004
...,...,...,...,...,...,...,...,...
63376,138301,115373,doctors,2014-10-22 22:19:24,201410,doctor,3.901503,66.0002
63377,138301,115373,French,2014-10-22 22:09:10,201410,french,3.988048,251.0000
63378,138301,115373,Louise Bourgoin,2014-10-22 22:09:19,201410,louise bourgoin,0.000100,0.0006
63379,138301,115373,love triangle,2014-10-22 22:19:50,201410,love triangle,4.084260,89.0002


And obtain the weighted mean of rating of the tags on the movies by month

In [37]:
wm = lambda x: np.average(x, weights=tags_weighted_rating.loc[x.index, "last_6_months_tag_count"])

tags_weighted_rating_agg = tags_weighted_rating\
.groupby(["movieId","year_month"])\
.agg(last_6_months_weighted_rating=("last_6_months_weighted_rating",wm))\
.reset_index()

If some movie has a weighted_raing low, it was because it didnt have a lot of tags, so we will substitute with the the mean of the variable in the month

In [38]:
tags_weighted_rating_agg.loc[tags_weighted_rating_agg.last_6_months_weighted_rating <= 0.001, 'last_6_months_weighted_rating'] = float("NaN")

tags_weighted_rating_agg = tags_weighted_rating_agg.groupby(['year_month'], sort=False)\
.apply(lambda x: x.fillna(x.mean()))\
.reset_index(drop = True)

In [39]:
tags_weighted_rating_agg[tags_weighted_rating_agg["movieId"]==1]

,movieId,year_month,last_6_months_weighted_rating
0,1,201405,3.827056
2121,1,201406,3.878078
3903,1,201408,3.891850
5383,1,201412,3.756539
7172,1,201501,3.847950
9116,1,201502,3.942630


## History variables from the ratings of the users

With the mean rating lets create the history of users ratings: 

* Last month mean users rating
* Last month count users rating
* Mean of the last 3 months of the rate given the users
* Mean of the last 6 months of the rate given the users
* The ratio of the last month mean genre rating and the mean of the last 3 months of the rate given the users
* The ratio of the last month mean genre rating and the mean of the last 6 months of the rate given the users

In [40]:

user_ratings_agg = ratings.groupby(["userId", "year_month"]).agg(count=("userId", "count"),
                                                                              mean_rating=("rating", "mean")).reset_index()

user_ratings_agg["last_1_months_user_rating"] = user_ratings_agg.groupby("userId")['mean_rating'].shift(1)\
            .reset_index()["mean_rating"]
user_ratings_agg["last_1_months_user_rating_counts"] = user_ratings_agg.groupby("userId")['count'].shift(1)\
            .reset_index()["count"]
user_ratings_agg["mean_3_months_user_rating"] = user_ratings_agg.groupby("userId")['last_1_months_user_rating'].rolling(3).mean()\
            .reset_index()["last_1_months_user_rating"]
user_ratings_agg["mean_6_months_user_rating"] = user_ratings_agg.groupby("userId")['last_1_months_user_rating'].rolling(6).mean()\
            .reset_index()["last_1_months_user_rating"]
user_ratings_agg["last_1_month_per_3_months_user_rating"] = user_ratings_agg.last_1_months_user_rating / \
            user_ratings_agg.mean_3_months_user_rating
user_ratings_agg["last_1_month_per_6_months_user_rating"] = user_ratings_agg.last_1_months_user_rating / \
            user_ratings_agg.mean_6_months_user_rating

In [41]:
user_ratings_agg.loc[user_ratings_agg["userId"]==55704]

,userId,year_month,count,mean_rating,last_1_months_user_rating,last_1_months_user_rating_counts,mean_3_months_user_rating,mean_6_months_user_rating,last_1_month_per_3_months_user_rating,last_1_month_per_6_months_user_rating
19384,55704,201301,4,3.375000,NaN,NaN,NaN,NaN,NaN,NaN
19385,55704,201302,6,3.083333,3.375000,4.0,NaN,NaN,NaN,NaN
19386,55704,201303,4,3.625000,3.083333,6.0,NaN,NaN,NaN,NaN
19387,55704,201304,2,3.750000,3.625000,4.0,3.361111,NaN,1.078512,NaN
19388,55704,201305,2,4.250000,3.750000,2.0,3.486111,NaN,1.075697,NaN
19389,55704,201306,5,3.300000,4.250000,2.0,3.875000,NaN,1.096774,NaN
19390,55704,201307,2,3.750000,3.300000,5.0,3.766667,3.563889,0.876106,0.925955
19391,55704,201308,29,3.706897,3.750000,2.0,3.766667,3.626389,0.995575,1.034087
19392,55704,201309,14,3.321429,3.706897,29.0,3.585632,3.730316,1.033820,0.993722
19393,55704,201310,5,3.800000,3.321429,14.0,3.592775,3.679721,0.924474,0.902631


In [42]:
user_ratings_agg = user_ratings_agg.groupby(['year_month'], sort=False)\
.apply(lambda x: x.fillna(x.mean()))\
.reset_index(drop = True)

In [43]:
user_ratings_agg = user_ratings_agg.loc[:,~user_ratings_agg.columns.isin(["count","mean_rating"])]
user_ratings_agg

,userId,year_month,last_1_months_user_rating,last_1_months_user_rating_counts,mean_3_months_user_rating,mean_6_months_user_rating,last_1_month_per_3_months_user_rating,last_1_month_per_6_months_user_rating
0,31,201502,3.540724,45.067735,3.533881,3.502598,0.989807,0.989026
1,96,201502,2.861538,65.000000,2.703846,2.775534,1.058321,1.030987
2,215,201502,2.625000,4.000000,3.152291,3.502598,0.832728,0.989026
3,279,201502,3.781250,16.000000,3.533881,3.502598,0.989807,0.989026
4,284,201502,3.540724,45.067735,3.533881,3.502598,0.989807,0.989026
...,...,...,...,...,...,...,...,...
47257,138070,201407,4.071429,14.000000,3.009921,3.527856,1.352670,0.998782
47258,138148,201407,2.833333,21.000000,3.204861,3.196181,0.884074,0.886475
47259,138280,201407,3.500000,1.000000,3.638889,3.831944,0.961832,0.913374
47260,138301,201407,2.944444,9.000000,3.346979,3.274888,0.879732,0.899098


##  Merging all the variables

In [77]:
ratings_to_join = ratings[ratings.year_month >= 201411]
ratings_to_join['target_var'] = 0
ratings_to_join.loc[ratings['rating']>= 4.0,"target_var"]=1
movies_ratings_to_join = movies_ratings[movies_ratings.year_month >= 201403]
movies_general_vars = movies.loc[:,~movies.columns.isin(["title","genres","year","genre_list"])]

In [78]:
final_dataset = ratings_to_join.merge(movies_genre_ratings_agg,"left")\
    .merge(tags_weighted_rating_agg,"left")\
    .merge(user_ratings_agg,"left")\
    .merge(movies_ratings_to_join,"left")\
    .merge(movies_general_vars,"left")

In [79]:
final_dataset

,userId,movieId,rating,timestamp,year_month,target_var,last_1_months_genre_rating,mean_3_months_genre_rating,mean_6_months_genre_rating,last_1_month_per_3_months_genre_rating,...,Children,Action,Western,Fantasy,Musical,Horror,Mystery,War,Thriller,Film-Noir
0,31,1,3.0,2015-02-23 23:18:07,201502,0,3.116378,3.146666,3.242472,0.990781,...,1,0,0,1,0,0,0,0,0,0
1,31,110,5.0,2015-02-23 23:17:53,201502,1,3.284891,3.343781,3.400271,0.982860,...,0,1,0,0,0,0,0,1,0,0
2,31,260,5.0,2015-02-23 23:17:13,201502,1,3.106176,3.124029,3.214912,0.994324,...,0,1,0,0,0,0,0,0,0,0
3,31,364,3.0,2015-02-25 06:13:27,201502,0,3.188801,3.238047,3.318860,0.985293,...,1,0,0,0,1,0,0,0,0,0
4,31,527,0.5,2015-02-23 23:19:58,201502,0,3.383728,3.465100,3.499553,0.976535,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442673,138468,109487,5.0,2014-12-22 13:35:37,201412,1,3.177909,3.268067,3.286991,0.972280,...,0,0,0,0,0,0,0,0,0,0
442674,138468,111759,4.0,2014-12-22 13:35:47,201412,1,3.152961,3.255481,3.280624,0.968390,...,0,1,0,0,0,0,0,0,0,0
442675,138468,112556,5.0,2014-12-22 13:35:39,201412,1,3.386351,3.408663,3.404156,0.993150,...,0,0,0,0,0,0,0,0,1,0
442676,138468,118696,3.5,2014-12-22 13:36:23,201412,0,3.170669,3.287278,3.309807,0.964519,...,0,0,0,1,0,0,0,0,0,0


In [80]:
final_dataset.dtypes

userId                                      int64
movieId                                     int64
rating                                    float64
timestamp                                  object
year_month                                  int64
target_var                                  int64
last_1_months_genre_rating                float64
mean_3_months_genre_rating                float64
mean_6_months_genre_rating                float64
last_1_month_per_3_months_genre_rating    float64
last_1_month_per_6_months_genre_rating    float64
last_6_months_weighted_rating             float64
last_1_months_user_rating                 float64
last_1_months_user_rating_counts          float64
mean_3_months_user_rating                 float64
mean_6_months_user_rating                 float64
last_1_month_per_3_months_user_rating     float64
last_1_month_per_6_months_user_rating     float64
last_1_months_movie_rating                float64
last_1_months_movie_rating_counts         float64
